In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append("../")

In [3]:
import pandas as pd
from GWA2019 import word_pos_utils as word_pos

In [4]:
moe_inst = word_pos.MOE("../resources/dict-revised.json")

# query_word_pos test

In [5]:
word_pos.query_word_pos("心跳", moe_inst)

['N', 'V']

In [6]:
word_pos.query_word_pos("企圖", moe_inst)

['V', 'N']

In [7]:
word_pos.query_word_pos("衝撞", moe_inst)

['V', 'V']

In [8]:
word_pos.query_word_pos("風景", moe_inst)

['N', 'N']

# apply the heuristics to word pair data

In [9]:
hyper_pairs = pd.read_csv("../data/merged_checked.csv")
hyper_pairs.drop("statement", axis = 1, inplace=True)

In [10]:
def query_word_pos_wrapper(x):
    ret = word_pos.query_word_pos(x, moe_inst)
    if any([x is None for x in ret]):
        return "--"
    else:
        return "".join(ret)
hyper_pairs["struct"] = hyper_pairs.lemma.apply(query_word_pos_wrapper)

In [11]:
word_struct = hyper_pairs.iloc[:,[4,0,1,2,3]]

In [12]:
word_pos.query_word_pos("一些", moe_inst)

['N', None]

In [13]:
word_struct.to_csv("../data/annot_word_struct/word_struct.csv")
## add a copy used to annotate
word_struct.to_csv("../data/annot_word_struct/annot_word_struct.csv")

In [14]:
stat_word_struct = word_struct[["struct", "response"]].groupby(["struct", "response"]).size()
stat_word_struct = stat_word_struct.reset_index().pivot(index="struct", columns="response")

In [15]:
stat_word_struct.columns = stat_word_struct.columns.droplevel(0)

In [16]:
stat_word_struct["sum"] = stat_word_struct.sum(1)

In [17]:
stat_word_struct["prop"] = stat_word_struct.iloc[:,1] / stat_word_struct["sum"]
stat_word_struct

response,0,1,sum,prop
struct,,,,
--,204,118,322,0.366460
NN,427,311,738,0.421409
NV,153,94,247,0.380567
VN,203,100,303,0.330033
VV,123,82,205,0.400000
